# WeatherDiffusion UNet Fine-tuning


In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA (NVIDIA GPU) is available.")
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  Device {i} Name: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA (NVIDIA GPU) is not available.")

CUDA (NVIDIA GPU) is available.
Number of CUDA devices: 1
  Device 0 Name: Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
target_dir = '/content/drive/MyDrive/GenImage'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)
    print(f"Created directory: {target_dir}")
os.chdir(target_dir)
print(f"Current directory: {os.getcwd()}")

Current directory: /content/drive/MyDrive/GenImage


In [ ]:
!git clone https://github.com/Trongnhat191/weatherdiff.git
%cd weatherdiff/WeatherDiffusion

fatal: destination path 'weatherdiff' already exists and is not an empty directory.
/content/drive/MyDrive/GenImage/weatherdiff/WeatherDiffusion


In [ ]:
!pip install pyyaml tensorboard einops pillow scikit-image

# Install Dependencies

In [ ]:
!ls -la

calculate_psnr_ssim.py  haze_data/  test/               weatherdiff.ipynb
configs/                LICENSE     train/
datasets/               models/     train_diffusion.py
eval_diffusion.py       README.md   utils/


# Data prepare

In [ ]:
!wget https://igi-web.tugraz.at/download/OzdenizciLegensteinTPAMI2023/WeatherDiff64.pth.tar
print("Pretrained model downloaded successfully!")

# Download Pretrained Model

In [ ]:
!rm -rf haze_data
!mkdir -p haze_data
%cd haze_data
!wget http://www.vision.ee.ethz.ch/ntire18/o-haze/O-HAZE.zip
print("Dataset downloaded!")

/content/drive/MyDrive/GenImage/weatherdiff/WeatherDiffusion/haze_data
--2025-11-25 04:47:31--  http://www.vision.ee.ethz.ch/ntire18/o-haze/O-HAZE.zip
Resolving www.vision.ee.ethz.ch (www.vision.ee.ethz.ch)... 129.132.19.216, 2001:67c:10ec:254::216
Connecting to www.vision.ee.ethz.ch (www.vision.ee.ethz.ch)|129.132.19.216|:80... connected.
HTTP request sent, awaiting response... 301 
Location: https://data.vision.ee.ethz.ch/cvl/ntire18//o-haze/O-HAZE.zip [following]
--2025-11-25 04:47:32--  https://data.vision.ee.ethz.ch/cvl/ntire18//o-haze/O-HAZE.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 573320339 (547M) [application/zip]
Saving to: ‘O-HAZE.zip’

O-HAZE.zip          100%[===================>] 546.76M  18.9MB/s    in 31s     

2025-11-25 04:48:04 (17.5 MB/s) - ‘O-HAZE.zip’ sa

In [ ]:
!apt-get install -y unzip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
!unzip -q O-HAZE.zip
print("Dataset extracted!")

Archive:  O-HAZE.zip
   creating: # O-HAZY NTIRE 2018/
   creating: # O-HAZY NTIRE 2018/GT/
 extracting: # O-HAZY NTIRE 2018/GT/01_outdoor_GT.jpg  
  inflating: # O-HAZY NTIRE 2018/GT/02_outdoor_GT.jpg  
  inflating: # O-HAZY NTIRE 2018/GT/03_outdoor_GT.JPG  
  inflating: # O-HAZY NTIRE 2018/GT/04_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/05_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/06_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/07_outdoor_GT.jpg  
  inflating: # O-HAZY NTIRE 2018/GT/08_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/09_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/10_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/11_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/12_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/13_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/14_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/15_outdoor_GT.jpg  
 extracting: # O-HAZY NTIRE 2018/GT/16_outdoor_GT.jpg  
 extracting:

In [ ]:
%cd '# O-HAZY NTIRE 2018'
print(f"Current directory: {os.getcwd()}")

/content/drive/MyDrive/GenImage/weatherdiff/WeatherDiffusion/haze_data/# O-HAZY NTIRE 2018


In [ ]:
import os
import re

folders_to_process = ["./GT/", "./hazy/"]

# Loop through each folder in the list
for folder in folders_to_process:
    # Ensure the folder exists before trying to list its contents
    if not os.path.exists(folder):
        print(f"Skipping folder: {folder} (Does not exist)")
        continue

    print(f"--- Processing folder: {folder} ---")

    for f in os.listdir(folder):
        lower_name = f.lower()
        main_name = None

        # Check for 'gt' pattern
        if '_gt' in lower_name:
            # Splits the name using '_gt' and joins the parts back
            main_name = re.split('_gt', lower_name)[0] + re.split('_gt', lower_name)[1]

        # Check for 'hazy' pattern
        elif '_hazy' in lower_name:
            # Splits the name using '_hazy' and joins the parts back
            main_name = re.split('_hazy', lower_name)[0] + re.split('_hazy', lower_name)[1]

        # Only rename if a pattern was matched and main_name was set
        if main_name:
            old_path = os.path.join(folder, f)
            new_path = os.path.join(folder, main_name)

            # Only rename if the new name is different from the old name
            if old_path != new_path:
                try:
                    os.rename(old_path, new_path)
                    print(f"Renamed: {f} -> {main_name}")
                except Exception as e:
                    print(f"Error renaming {f}: {e}")
            else:
                # This handles cases where f.lower() is already the main_name
                # and you would rename a file to itself.
                pass
        else:
            print(f"Skipping: {f} (No '_gt' or '_hazy' pattern found)")

--- Processing folder: ./GT/ ---
Renamed: 01_outdoor_GT.jpg -> 01_outdoor.jpg
Renamed: 02_outdoor_GT.jpg -> 02_outdoor.jpg
Renamed: 03_outdoor_GT.JPG -> 03_outdoor.jpg
Renamed: 04_outdoor_GT.jpg -> 04_outdoor.jpg
Renamed: 05_outdoor_GT.jpg -> 05_outdoor.jpg
Renamed: 06_outdoor_GT.jpg -> 06_outdoor.jpg
Renamed: 07_outdoor_GT.jpg -> 07_outdoor.jpg
Renamed: 08_outdoor_GT.jpg -> 08_outdoor.jpg
Renamed: 09_outdoor_GT.jpg -> 09_outdoor.jpg
Renamed: 10_outdoor_GT.jpg -> 10_outdoor.jpg
Renamed: 11_outdoor_GT.jpg -> 11_outdoor.jpg
Renamed: 12_outdoor_GT.jpg -> 12_outdoor.jpg
Renamed: 13_outdoor_GT.jpg -> 13_outdoor.jpg
Renamed: 14_outdoor_GT.jpg -> 14_outdoor.jpg
Renamed: 15_outdoor_GT.jpg -> 15_outdoor.jpg
Renamed: 16_outdoor_GT.jpg -> 16_outdoor.jpg
Renamed: 17_outdoor_GT.jpg -> 17_outdoor.jpg
Renamed: 18_outdoor_GT.jpg -> 18_outdoor.jpg
Renamed: 19_outdoor_GT.jpg -> 19_outdoor.jpg
Renamed: 20_outdoor_GT.jpg -> 20_outdoor.jpg
Renamed: 21_outdoor_GT.JPG -> 21_outdoor.jpg
Renamed: 22_outdoor_GT

In [ ]:
%cd ..
print(f"Current directory: {os.getcwd()}")

/content/drive/MyDrive/GenImage/weatherdiff/WeatherDiffusion/haze_data


In [ ]:
!mkdir -p train/GT train/hazy test/GT test/hazy
print("Train and test directories created!")

In [ ]:
folder = os.path.join('# O-HAZY NTIRE 2018',"./hazy/")
print(folder)
image_names = []
for f in os.listdir(folder):
    image_names.append(f)
image_names.sort()
print(image_names)

# O-HAZY NTIRE 2018/./hazy/
['01_outdoor.jpg', '02_outdoor.jpg', '03_outdoor.jpg', '04_outdoor.jpg', '05_outdoor.jpg', '06_outdoor.jpg', '07_outdoor.jpg', '08_outdoor.jpg', '09_outdoor.jpg', '10_outdoor.jpg', '11_outdoor.jpg', '12_outdoor.jpg', '13_outdoor.jpg', '14_outdoor.jpg', '15_outdoor.jpg', '16_outdoor.jpg', '17_outdoor.jpg', '18_outdoor.jpg', '19_outdoor.jpg', '20_outdoor.jpg', '21_outdoor.jpg', '22_outdoor.jpg', '23_outdoor.jpg', '24_outdoor.jpg', '25_outdoor.jpg', '26_outdoor.jpg', '27_outdoor.jpg', '28_outdoor.jpg', '29_outdoor.jpg', '30_outdoor.jpg', '31_outdoor.jpg', '32_outdoor.jpg', '33_outdoor.jpg', '34_outdoor.jpg', '35_outdoor.jpg', '36_outdoor.jpg', '37_outdoor.jpg', '38_outdoor.jpg', '39_outdoor.jpg', '40_outdoor.jpg', '41_outdoor.jpg', '42_outdoor.jpg', '43_outdoor.jpg', '44_outdoor.jpg', '45_outdoor.jpg']


In [ ]:
import shutil

for image_name in image_names[:40]:
    #move GT images
    src = os.path.join("# O-HAZY NTIRE 2018","GT", image_name)
    dst = os.path.join('train', 'GT')
    shutil.move(src, dst)
    #move hazy images
    src = os.path.join("# O-HAZY NTIRE 2018","hazy", image_name)
    dst = os.path.join('train', 'hazy')
    shutil.move(src, dst)

for image_name in image_names[40:]:
    #move GT images
    src = os.path.join("# O-HAZY NTIRE 2018","GT", image_name)
    dst = os.path.join('test', 'GT')
    shutil.move(src, dst)
    #move hazy images
    src = os.path.join("# O-HAZY NTIRE 2018","hazy", image_name)
    dst = os.path.join('test', 'hazy')
    shutil.move(src, dst)

print(f"Moved {len(image_names[:40])} images to train and {len(image_names[40:])} images to test")

# Train

In [ ]:
import os

print("Directory structure:")
print(f"Train GT images: {len(os.listdir('train/GT'))}")
print(f"Train hazy images: {len(os.listdir('train/hazy'))}")
print(f"Test GT images: {len(os.listdir('test/GT'))}")
print(f"Test hazy images: {len(os.listdir('test/hazy'))}")
print(f"\nCurrent directory: {os.getcwd()}")

# Verify Data Structure

In [ ]:
%cd ..
print(f"Current directory: {os.getcwd()}")

/content/drive/MyDrive/GenImage/weatherdiff/WeatherDiffusion


In [ ]:
import yaml
import os

# Get the current working directory (should be WeatherDiffusion)
base_dir = os.getcwd()
print(f"Base directory: {base_dir}")

# Create config for O-HAZE fine-tuning
config = {
    'data': {
        'dataset': 'OHaze',
        'image_size': 64,
        'channels': 3,
        'num_workers': 4,
        'data_dir': base_dir,
        'conditional': True
    },
    'model': {
        'in_channels': 3,
        'out_ch': 3,
        'ch': 128,
        'ch_mult': [1, 2, 3, 4],
        'num_res_blocks': 2,
        'attn_resolutions': [16],
        'dropout': 0.0,
        'ema_rate': 0.999,
        'ema': True,
        'resamp_with_conv': True
    },
    'diffusion': {
        'beta_schedule': 'linear',
        'beta_start': 0.0001,
        'beta_end': 0.02,
        'num_diffusion_timesteps': 1000
    },
    'training': {
        'patch_n': 16,
        'batch_size': 4,  # Reduced for Colab
        'n_epochs': 500,
        'n_iters': 100000,
        'snapshot_freq': 2000,  # Save more frequently
        'validation_freq': 1000
    },
    'sampling': {
        'batch_size': 2,  # Reduced for Colab
        'last_only': True
    },
    'optim': {
        'weight_decay': 0.0,
        'optimizer': 'Adam',
        'lr': 0.00002,
        'amsgrad': False,
        'eps': 1e-8
    }
}

# Save config file
config_path = os.path.join('configs', 'ohaze.yml')
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"Config file created at: {config_path}")
print("\nConfig contents:")
print(yaml.dump(config, default_flow_style=False))

# Create Config File for O-HAZE

In [ ]:
!python train_diffusion.py \
--config "ohaze.yml" \
--resume "WeatherDiff64.pth.tar" \
--sampling_timesteps 25 \
--image_folder "results/images/OHaze/train/"

Using device: cuda
=> using dataset 'MyDataset'
Data Loaded!
=> creating denoising-diffusion model...
Found 40 images in ./haze_data/train/hazy
Found 5 images in ./haze_data/test/hazy
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-05
    maximize: False
    weight_decay: 0.001
)
Current learning rate before training: 1e-05
epoch:  0
step: 10, loss: 10574.92578125, data time: 1.27

In [ ]:
# Run evaluation (replace checkpoint path with your trained model)
# !python test_ohaze.py --checkpoint "exp/logs/your_checkpoint.pth.tar"

In [ ]:
# Find the latest checkpoint
import glob
checkpoints = glob.glob('exp/logs/*.pth.tar')
if checkpoints:
    latest_checkpoint = max(checkpoints, key=os.path.getctime)
    print(f"Latest checkpoint: {latest_checkpoint}")
else:
    print("No checkpoints found yet")

# Evaluate the Fine-tuned Model

After training, you can evaluate the model on the test set.

In [ ]:
# Check training outputs
!ls -lh exp/logs/
!ls -lh results/images/OHaze/train/ 2>/dev/null || echo "No validation images yet"

# Monitor Training (Optional)

You can monitor the training progress by checking the saved checkpoints and validation images in the results folder. The model will save checkpoints every 2000 iterations.